In [33]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import MultiPoint
import folium
from folium.plugins import HeatMap, HeatMapWithTime
import gpxpy
import glob

%matplotlib inline

In [2]:
gpx_file = open('cycle-20200525T102721.gpx', 'r')
gpx = gpxpy.parse(gpx_file)
# check that data is essentially all in .points attribute of first segment and first track
print(len(gpx.tracks))
print(len(gpx.tracks[0].segments))
print(len(gpx.tracks[0].segments[0].points))

data = gpx.tracks[0].segments[0].points

## Start Position
print(data[0])
## End Position
print(data[-1])

# put into dataframe
df = pd.DataFrame(columns=['lon', 'lat', 'alt', 'time'])
for point in data:
    df = df.append({'lon': point.longitude, 'lat' : point.latitude, 'alt' : point.elevation, 'time' : point.time}, ignore_index=True)
df.head(2)

1
1
4494
[trkpt:53.661908,-1.836951@254.6@2020-05-25 09:27:20+00:00]
[trkpt:53.661752,-1.836644@253.8@2020-05-25 10:42:13+00:00]


,lon,lat,alt,time
0,-1.836951,53.661908,254.6,2020-05-25 09:27:20+00:00
1,-1.836973,53.661924,254.7,2020-05-25 09:27:21+00:00


In [4]:
# put into geodataframe
gdf = gpd.GeoDataFrame(data=df,
                       geometry=gpd.points_from_xy(df.lon, df.lat))
gdf.crs = 'epsg:4326'
print(gdf.crs)
gdf = gdf.to_crs(epsg=27700)
gdf.head(2)

epsg:4326


,lon,lat,alt,time,geometry
0,-1.836951,53.661908,254.6,2020-05-25 09:27:20+00:00,POINT (410872.023 418392.484)
1,-1.836973,53.661924,254.7,2020-05-25 09:27:21+00:00,POINT (410870.565 418394.261)


In [31]:
centre = (54.083797, -2.858426)
m = folium.Map(location=centre, zoom_start=6, width='40%')#, height='80%')

heat_data = gdf[['lat', 'lon']].dropna().drop_duplicates()
heat_data = [[row['lat'],row['lon']] for index, row in heat_data.iterrows()]
# Plot it on the map
# HeatMap(heat_data, radius=10, blur=15, min_opacity=.4).add_to(m)
# m

# Viz with time
# heat_data_by_bin = []
# df['binned_time'] = df['time'].dt.floor('2min')
# for i,group_df in enumerate(df.sort_values('time').groupby('binned_time')):
#     group_df = group_df[1]
#     heat_data = group_df[['lat', 'lon']].dropna()   #.drop_duplicates()
#     heat_data = [[row['lat'],row['lon']] for index, row in heat_data.iterrows()]
#     if i > 0:
#         heat_data = heat_data_by_bin[i-1] + heat_data
#     heat_data_by_bin.append(heat_data)
# # Plot it on the map
# HeatMapWithTime(heat_data_by_bin, radius=7, auto_play=True).add_to(m)
# m

# Multiple files

In [47]:
df_all = []

files = glob.glob('*gpx')
for filename in files:
    gpx_file = open(filename, 'r')
    gpx = gpxpy.parse(gpx_file)
    # check that data is all in .points attribute of first segment and first track
    if (len(gpx.tracks) != 1) | (len(gpx.tracks[0].segments) != 1):
        print(filename)
        print(len(gpx.tracks))
        df = pd.DataFrame(columns=['lon', 'lat', 'alt', 'time'])
        for i in range(len(gpx.tracks)):
            print(len(gpx.tracks[i].segments))
            for j in range(len(gpx.tracks[0].segments)):
                print(len(gpx.tracks[i].segments[j].points))
                data = gpx.tracks[i].segments[j].points
                for point in data:
                    df = df.append({'lon':point.longitude,
                                    'lat':point.latitude,
                                    'alt':point.elevation,
                                    'time':point.time},
                                   ignore_index=True)
        df['binned_time'] = df['time'].dt.floor('1min')
        df = df.groupby('binned_time').agg({'lon':'mean', 'lat':'mean', 'alt':'mean'}).reset_index()
        df_all.append(df)
    
    else:
        data = gpx.tracks[0].segments[0].points
        # put into dataframe
        df = pd.DataFrame(columns=['lon', 'lat', 'alt', 'time'])
        for point in data:
            df = df.append({'lon':point.longitude,
                            'lat':point.latitude,
                            'alt':point.elevation,
                            'time':point.time},
                           ignore_index=True)
        # group into 1 minute bins
        df['binned_time'] = df['time'].dt.floor('1min')
        df = df.groupby('binned_time').agg({'lon':'mean', 'lat':'mean', 'alt':'mean'}).reset_index()
        df_all.append(df)

df_all = pd.concat(df_all)

cycle-20200620T105509.gpx
1
2
6260
6260
3498
3498
run-20200616T070457.gpx
1
2
2943
2943
1
1


In [48]:
centre = (54.083797, -2.858426)
m = folium.Map(location=centre, zoom_start=6, width='40%')#, height='80%')

heat_data = df_all[['lat', 'lon']].dropna().drop_duplicates()
heat_data = [[row['lat'],row['lon']] for index, row in heat_data.iterrows()]
# Plot it on the map
HeatMap(heat_data, radius=10, blur=15, min_opacity=.4).add_to(m)
m

## test with functions

In [3]:
import glob
import functions

files = glob.glob('../*gpx')
df = functions.read_data(files)
m = functions.make_heatmap(df)

m